#### Roll Up and Drill Down

## Roll-up
- Stepping up the level of aggregation to a large grouping
- e.g.`city` is summed as `country`

TODO: Write a query that calculates revenue (sales_amount) by day, rating, and country. Sort the data by revenue in descending order, and limit the data to the top 20 results. The first few rows of your output should match the table below.

In [ ]:
# Mine
%%time
%%sql
SELECT dt.day as day, dv.rating, ds.country, sum(fs.sales_amount) as revenue
FROM dimdate dt 
JOIN factsales fs ON (dt.date_key = fs.date_key)
JOIN dimmovie dv ON (dv.movie_key = fs.movie_key)
JOIN dimstore ds ON (ds.store_key = fs.store_key)
GROUP BY day, dv.rating, ds.country
ORDER BY revenue DESC
LIMIT 20;

#### Drill down

## Drill-down
- Breaking up one of the dimensions to a lower level.
- e.g.`city` is broken up into  `districts`

TODO: Write a query that calculates revenue (sales_amount) by day, rating, and district. Sort the data by revenue in descending order, and limit the data to the top 20 results. The first few rows of your output should match the table below.

In [ ]:
# Mine
%%time
%%sql
SELECT dt.day as day, dv.rating, ds.district, sum(fs.sales_amount) as revenue
FROM dimdate dt 
JOIN factsales fs ON (dt.date_key = fs.date_key)
JOIN dimmovie dv ON (dv.movie_key = fs.movie_key)
JOIN dimstore ds ON (ds.store_key = fs.store_key)
GROUP BY day, dv.rating, ds.district
ORDER BY revenue DESC
LIMIT 20;

#### Slicing and Dicing

# Start with a simple cube
TODO: Write a query that calculates the revenue (sales_amount) by day, rating, and city. Remember to join with the appropriate dimension tables to replace the keys with the dimension labels. Sort by revenue in descending order and limit to the first 20 rows. The first few rows of your output should match the table below.

In [ ]:
# Mine
%%time
%%sql
SELECT dt.day as day, dv.rating, ds.city, sum(fs.sales_amount) as revenue
FROM dimdate dt 
JOIN factsales fs ON (dt.date_key = fs.date_key)
JOIN dimmovie dv ON (dv.movie_key = fs.movie_key)
JOIN dimstore ds ON (ds.store_key = fs.store_key)
GROUP BY day, dv.rating, ds.city
ORDER BY revenue DESC
LIMIT 20;

#### Slicing

Slicing is the reduction of the dimensionality of a cube by 1 e.g. 3 dimensions to 2, fixing one of the dimensions to a single value. In the example above, we have a 3-dimensional cube on day, rating, and country.

TODO: Write a query that reduces the dimensionality of the above example by limiting the results to only include movies with a `rating` of "PG-13". Again, sort by revenue in descending order and limit to the first 20 rows. The first few rows of your output should match the table below. 

In [ ]:
# Mine
%%time
%%sql
SELECT dt.day as day, dv.rating, ds.city, sum(fs.sales_amount) as revenue
FROM dimdate dt 
JOIN factsales fs ON (dt.date_key = fs.date_key)
JOIN dimmovie dv ON (dv.movie_key = fs.movie_key)
JOIN dimstore ds ON (ds.store_key = fs.store_key)
WHERE dv.rating = 'PG-13'
GROUP BY day, dv.rating, ds.city
ORDER BY revenue DESC
LIMIT 20;

## Dicing
Dicing is creating a subcube with the same dimensionality but fewer values for  two or more dimensions. 

TODO: Write a query to create a subcube of the initial cube that includes moves with:
* ratings of PG or PG-13
* in the city of Bellevue or Lancaster
* day equal to 1, 15, or 30

The first few rows of your output should match the table below. 

In [ ]:
# Mine
%%time
%%sql
SELECT dt.day as day, dv.rating, ds.city, sum(fs.sales_amount) as revenue
FROM dimdate dt 
JOIN factsales fs ON (dt.date_key = fs.date_key)
JOIN dimmovie dv ON (dv.movie_key = fs.movie_key)
JOIN dimstore ds ON (ds.store_key = fs.store_key)
WHERE dv.rating = 'PG-13' OR dv.rating = 'PG' AND
ds.city = 'Lancaster' OR ds.city = 'Bellevue' 
AND dt.day = 1 OR dt.day = 15 OR dt.day = 30
GROUP BY day, dv.rating, ds.city
ORDER BY revenue DESC
LIMIT 20;

In [ ]:
# Mine
%%time
%%sql
SELECT dt.day as day, dv.rating, ds.city, sum(fs.sales_amount) as revenue
FROM dimdate dt 
JOIN factsales fs ON (dt.date_key = fs.date_key)
JOIN dimmovie dv ON (dv.movie_key = fs.movie_key)
JOIN dimstore ds ON (ds.store_key = fs.store_key)
WHERE dv.rating in ('PG-13', 'PG') AND
ds.city in ('Lancaster', 'Bellevue') AND
dt.day in ('1','15','30')
GROUP BY day, dv.rating, ds.city
ORDER BY revenue DESC
LIMIT 20;

#### Grouping Sets

# Grouping Sets
- It happens often that for 3 dimensions, you want to aggregate a fact:
    - by nothing (total)
    - then by the 1st dimension
    - then by the 2nd 
    - then by the 3rd 
    - then by the 1st and 2nd
    - then by the 2nd and 3rd
    - then by the 1st and 3rd
    - then by the 1st and 2nd and 3rd
    
- Since this is very common, and in all cases, we are iterating through all the fact table anyhow, there is a more clever way to do that using the SQL grouping statement "GROUPING SETS" 

## Total Revenue

TODO: Write a query that calculates total revenue (sales_amount)

In [ ]:
# Mine
%%sql
SELECT sum(sales_amount) as total_revenue
FROM factsales

## Revenue by Country
TODO: Write a query that calculates total revenue (sales_amount) by country

In [ ]:
# Mine
%%sql
SELECT ds.country, sum(fs.sales_amount) as revenue
FROM factsales fs 
JOIN dimstore ds ON (ds.store_key = fs.store_key)
GROUP BY ds.country
LIMIT 20;

## Revenue by Month
TODO: Write a query that calculates total revenue (sales_amount) by month

In [ ]:
# Mine
%%sql
SELECT dt.month, sum(fs.sales_amount) as revenue
FROM dimdate dt  
JOIN factsales fs ON (dt.date_key = fs.date_key)
GROUP BY dt.month
LIMIT 20;

## Revenue by Month & Country
TODO: Write a query that calculates total revenue (sales_amount) by month and country. Sort the data by month, country, and revenue in descending order. The first few rows of your output should match the table below.

In [ ]:
# Mine
%%sql
SELECT dt.month, ds.country, sum(fs.sales_amount) as revenue
FROM dimdate dt 
JOIN factsales fs ON (dt.date_key = fs.date_key)
JOIN dimstore ds ON (ds.store_key = fs.store_key)
GROUP BY (dt.month, ds.country)
ORDER BY dt.month, ds.country, revenue DESC
LIMIT 20;

## Revenue Total, by Month, by Country, by Month & Country All in one shot

TODO: Write a query that calculates total revenue at the various grouping levels done above (total, by month, by country, by month & country) all at once using the grouping sets function. Your output should match the table below.

In [ ]:
%%sql
SELECT dt.month, ds.country, sum(fs.sales_amount) as revenue
FROM dimdate dt 
JOIN factsales fs ON (dt.date_key = fs.date_key)
JOIN dimstore ds ON (ds.store_key = fs.store_key)
GROUP BY grouping sets((), dt.month, ds.country, (dt.month, ds.country));

#### OLAP Cubes

# CUBE 
- Group by CUBE (dim1, dim2, ..) , produces all combinations of different lenghts in one go.
- This view could be materialized in a view and queried which would save lots repetitive aggregations

TODO: Write a query that calculates the various levels of aggregation done in the grouping sets exercise (total, by month, by country, by month & country) using the CUBE function. Your output should match the table below.


In [ ]:
%%sql
SELECT dt.month, ds.country, sum(fs.sales_amount) as revenue
FROM dimdate dt 
JOIN factsales fs ON (dt.date_key = fs.date_key)
JOIN dimstore ds ON (ds.store_key = fs.store_key)
GROUP BY cube(dt.month, ds.country);

## Revenue Total, by Month, by Country, by Month & Country All in one shot, NAIVE way
The naive way to create the same table as above is to write several queries and UNION them together. Grouping sets and cubes produce queries that are shorter to write, easier to read, and more performant. Run the naive query below and compare the time it takes to run to the time it takes the cube query to run.

In [ ]:
%%time
%%sql
SELECT  NULL as month, NULL as country, sum(sales_amount) as revenue
FROM factSales
    UNION all 
SELECT NULL, dimStore.country,sum(sales_amount) as revenue
FROM factSales
JOIN dimStore on (dimStore.store_key = factSales.store_key)
GROUP by  dimStore.country
    UNION all 
SELECT cast(dimDate.month as text) , NULL, sum(sales_amount) as revenue
FROM factSales
JOIN dimDate on (dimDate.date_key = factSales.date_key)
GROUP by dimDate.month
    UNION all
SELECT cast(dimDate.month as text),dimStore.country,sum(sales_amount) as revenue
FROM factSales
JOIN dimDate     on (dimDate.date_key         = factSales.date_key)
JOIN dimStore on (dimStore.store_key = factSales.store_key)
GROUP by (dimDate.month, dimStore.country)